In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [27]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import itertools
import matplotlib.pyplot as plt
import glob
from PIL import Image
import shutil
import os
from torch.utils.data import Dataset, DataLoader
from pathlib import Path

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
transforms = {'train': transforms.Compose([
    transforms.Resize(224),
    transforms.RandomCrop(200),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.RandomCrop(200),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'eval': transforms.Compose([
        transforms.Resize(224),
        transforms.RandomCrop(200),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

In [5]:
class PlantSeedlingDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = Path(root_dir)
        self.x = []
        self.y = []
        self.transform = transform
        self.num_classes = 0
        if self.root_dir.name=='train':
            for i, _dir in enumerate(self.root_dir.glob('*')):
                for file in _dir.glob('*'):
                    self.x.append(file)
                    self.y.append(i)
                self.num_classes+=1
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        image = Image.open(self.x[index]).convert('RGB')
        if self.transform: image = self.transform(image)
        return image, self.y[index]

In [6]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 12)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=0.)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /tmp/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:02<00:00, 44194341.92it/s]


In [7]:
train_set = PlantSeedlingDataset(Path('../input/').joinpath('train'), transforms['train'])
eval_set = PlantSeedlingDataset(Path('../input/').joinpath('train'), transforms['train'])
train_set, val_set = torch.utils.data.dataset.random_split(train_set, [int((len(train_set)*4)/5), int(len(train_set)/5)])
train_dl = DataLoader(train_set, batch_size=8, shuffle=True, num_workers=0)
val_dl = DataLoader(val_set, batch_size=16, shuffle=False, num_workers=0)
eval_dl = DataLoader(eval_set, batch_size=16, shuffle=False, num_workers=0)

In [18]:
def score(model, dataloader):
    calc_loss = 0.
    calc_correct = 0.
    calc_count = 0.

    for data in dataloader:
        input, label = data
        input = Variable(input.cuda())
        label = Variable(label.cuda())
        output = model(input)
        _, pred = torch.max(output.data, 1)
        loss = criterion(output, label)
        calc_loss +=loss.item()
        calc_correct +=torch.sum(pred==label.data)
        calc_count+=output.data.shape[0]
    return round(calc_loss/calc_count, 4)

In [19]:
def train(train_loader, val_loader, eval_loader, epochs):
    for epoch in range(epochs):
        calc_loss = 0.
        calc_correct = 0.
        calc_count = 0.
        for data in train_loader:
            inputs, labels = data
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            calc_loss+=loss.item()
            calc_correct+=torch.sum(preds==labels.data)
            calc_count+=outputs.data.shape[0]
        print('-------------')
        print('epoch:', epoch)
        print('run loss:', round(calc_loss/calc_count, 4))
        print('train loss:', score(model, eval_loader))
        print('val loss:', score(model, val_loader))
    return model

In [20]:
def adjust_lr(optimizer, value):
    for param_group in optimizer.param_groups:
        param_group['lr'] = value
    return optimizer

In [21]:
model = train(train_dl, val_dl, eval_dl, 5)

-------------
epoch: 0
run loss: 0.0342
train loss: 0.2428
val loss: 0.0198
-------------
epoch: 1
run loss: 0.0319
train loss: 0.2463
val loss: 0.0133
-------------
epoch: 2
run loss: 0.0246
train loss: 0.2655
val loss: 0.0148
-------------
epoch: 3
run loss: 0.0261
train loss: 0.2649
val loss: 0.0138
-------------
epoch: 4
run loss: 0.0228
train loss: 0.2611
val loss: 0.0124


In [22]:
optimizer = adjust_lr(optimizer, 0.00005)
model = train(train_dl, val_dl, eval_dl, 5)

-------------
epoch: 0
run loss: 0.0138
train loss: 0.247
val loss: 0.0096
-------------
epoch: 1
run loss: 0.0109
train loss: 0.2536
val loss: 0.0074
-------------
epoch: 2
run loss: 0.0116
train loss: 0.2579
val loss: 0.0083
-------------
epoch: 3
run loss: 0.0107
train loss: 0.2483
val loss: 0.0082
-------------
epoch: 4
run loss: 0.0106
train loss: 0.2573
val loss: 0.0084


In [32]:
import pandas as pd

In [35]:
!ls ../

config	input  lib  working


In [39]:
def test():
    data_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    test_root = Path('../input/')
    classes = [_dir.name for _dir in test_root.joinpath('train').glob('*')]
    sample_submission = pd.read_csv(str(test_root.joinpath('sample_submission.csv')))
    submission = sample_submission.copy()
    for i, filename in enumerate(sample_submission['file']):
        image = Image.open(str(test_root.joinpath('test').joinpath(filename))).convert('RGB')
        image = data_transform(image).unsqueeze(0)
        inputs = Variable(image.cuda())
        output = model(inputs)
        _, preds = torch.max(output.data, 1)
        submission['species'][i] = classes[preds[0]]
    submission.to_csv('../working/submission.csv', index=False)

In [40]:
test()

In [43]:
df = pd.read_csv('../working/submission.csv')
df

,file,species
0,0021e90e4.png,Small-flowered Cranesbill
1,003d61042.png,Small-flowered Cranesbill
2,007b3da8b.png,Sugar beet
3,0086a6340.png,Common Chickweed
4,00c47e980.png,Common Chickweed
5,00d090cde.png,Common Chickweed
6,00ef713a8.png,Common Chickweed
7,01291174f.png,Common Chickweed
8,026716f9b.png,Common Chickweed
9,02cfeb38d.png,Common Chickweed


# References:
1. https://pytorch.org/tutorials/
2. https://github.com/magical2world/Plant-seedlings-recognition-through-fine-tuning-resnet18-using-pytorch
3. https://github.com/remorsecs/Kaggle-Plant-Seedlings-Classification-Example
4. https://github.com/brightertiger/kaggle/tree/master/playground/plant-seed